In [36]:
import random
import pandas as pd
import jieba
import re

In [219]:
#编程实践部分
#1. 设计你自己的句子生成器
#1.1.定义自己的语法

In [264]:
#语法一：甲方催图
owner = '''
owner = 量词 项目 原因 ， 时间 要求 
量词 = 这张 | 这套 
项目 = 图 | 规范书 | 图纸
原因 = 很急 | 全都不对 | 有问题
时间 = 今天下班前 | 今天加班 |一个小时后
要求 = 完成 | 改完 | 交给我
'''

In [265]:
#语法二：旅游网客服
agent = '''
agent = 寒暄 报数 询问 具体业务 结尾
寒暄 = 称谓 您好 | 您好
称谓 = 人称 ,
人称 = 先生 | 女士 
报数 = 我是 数字 号客服 ,
数字 = 单个数字 | 数字 单个数字
单个数字 = 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 
询问 = 请问您要 | 您需要
具体业务 = 具体业务1 | 具体业务2
具体业务1 = 订机票 | 订酒店 | 定制旅游
具体业务2 = 投诉 | 提建议 | 人工服务 | 兑现奖品
结尾 = 吗？'''

In [266]:
#把语法存储在一个字典里
def create_grammar(grammar_str, split='=', line_split='\n'):
    grammar = {}
    for line in grammar_str.split(line_split):
        if not line.strip(): continue
        exp, stmt = line.split(split)
        grammar[exp.strip()] = [s.split() for s in stmt.split('|')]
    return grammar

In [267]:
#生成句子模型
def generate(gram, target):
    if target not in gram: return target # means target is a terminal expression
    
    expanded = [generate(gram, t) for t in random.choice(gram[target])]
    return ''.join([e if e != '/n' else '\n' for e in expanded if e != 'null'])

In [271]:
#1.2 用generate函数生成随机句子
generate(gram=create_grammar(agent), target='agent')

'您好我是8号客服,请问您要投诉吗？'

In [272]:
#1.3 定义一个函数，generate_n，将generate扩展，使其能够生成n个句子:
def generate_n(gram,target,n):
    sens=[]
    for i in range(n):
        sens.append(generate(gram,target)) 
    return sens

In [277]:
generate_n(gram=create_grammar(agent), target='agent',n=5)

['您好我是42号客服,请问您要订酒店吗？',
 '您好我是417号客服,您需要提建议吗？',
 '您好我是4号客服,请问您要订酒店吗？',
 '您好我是3号客服,您需要提建议吗？',
 '女士,您好我是74号客服,请问您要定制旅游吗？']

In [27]:
#2. 使用新数据源完成语言模型的训练
import os
os.chdir(r'C:\Users\yuan-he\Desktop\【袁禾-人工智能与自然语言处理】培训课程编程能力测试及课程协议\第一课\Assignment')

In [278]:
#2.1 获取文本数据集
data = pd.read_csv('movie_comments.csv', encoding='utf-8',low_memory=False)

In [279]:
data.head()

,id,link,name,comment,star
0,1,https://movie.douban.com/subject/26363254/,战狼2,吴京意淫到了脑残的地步，看了恶心想吐,1
1,2,https://movie.douban.com/subject/26363254/,战狼2,首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮...,2
2,3,https://movie.douban.com/subject/26363254/,战狼2,吴京的炒作水平不输冯小刚，但小刚至少不会用主旋律来炒作…吴京让人看了不舒服，为了主旋律而主旋...,2
3,4,https://movie.douban.com/subject/26363254/,战狼2,凭良心说，好看到不像《战狼1》的续集，完虐《湄公河行动》。,4
4,5,https://movie.douban.com/subject/26363254/,战狼2,中二得很,1


In [281]:
comment=data['comment'].tolist()

In [282]:
#数据清洗
def token(string):
    # we will learn the regular expression next course.
    return re.findall('\w+', string)

In [283]:
#进行文本清洗，获得所有的纯文本
comment_clean = [''.join(token(str(a)))for a in comment]
comment_clean[:5]

['吴京意淫到了脑残的地步看了恶心想吐',
 '首映礼看的太恐怖了这个电影不讲道理的完全就是吴京在实现他这个小粉红的英雄梦各种装备轮番上场视物理逻辑于不顾不得不说有钱真好随意胡闹',
 '吴京的炒作水平不输冯小刚但小刚至少不会用主旋律来炒作吴京让人看了不舒服为了主旋律而主旋律为了煽情而煽情让人觉得他是个大做作大谎言家729更新片子整体不如湄公河行动1整体不够流畅编剧有毒台词尴尬2刻意做作的主旋律煽情显得如此不合时宜而又多余',
 '凭良心说好看到不像战狼1的续集完虐湄公河行动',
 '中二得很']

In [284]:
with open('douban_comment.txt', 'w',encoding='utf-8') as f:
    for a in comment_clean:
        f.write(a + '\n')

In [285]:
len(comment_clean)

261497

In [286]:
def cut(string): return list(jieba.cut(string))

In [288]:
#将这些文本进行切词
TOKEN=[]
for i, line in enumerate((open('douban_comment.txt',encoding='utf-8'))):   
    if i % 20000 == 0: print(i)
    TOKEN += cut(line)

0
20000
40000
60000
80000
100000
120000
140000
160000
180000
200000
220000
240000
260000


In [290]:
from collections import Counter

In [291]:
words_count = Counter(TOKEN)

In [292]:
words_count.most_common(50)

[('的', 328262),
 ('\n', 261497),
 ('了', 102420),
 ('是', 73106),
 ('我', 50338),
 ('都', 36255),
 ('很', 34712),
 ('看', 34022),
 ('电影', 33675),
 ('也', 32065),
 ('和', 31290),
 ('在', 31245),
 ('不', 28435),
 ('有', 27939),
 ('就', 25685),
 ('人', 23909),
 ('好', 22858),
 ('啊', 20803),
 ('这', 17484),
 ('还', 17449),
 ('一个', 17343),
 ('你', 17282),
 ('还是', 16425),
 ('但', 15578),
 ('故事', 15010),
 ('没有', 14343),
 ('就是', 14007),
 ('喜欢', 13566),
 ('让', 13304),
 ('太', 12676),
 ('又', 11566),
 ('剧情', 11359),
 ('没', 10858),
 ('说', 10764),
 ('吧', 10747),
 ('他', 10675),
 ('不错', 10416),
 ('得', 10349),
 ('到', 10341),
 ('给', 10300),
 ('这个', 10058),
 ('上', 10054),
 ('被', 9939),
 ('对', 9824),
 ('最后', 9694),
 ('一部', 9693),
 ('片子', 9590),
 ('什么', 9571),
 ('能', 9532),
 ('与', 9168)]

In [293]:
#送入之前定义的语言模型中，判断文本的合理程度

In [294]:
TOKEN = [str(t) for t in TOKEN]

In [296]:
TOKEN_2_GRAM = [''.join(TOKEN[i:i+2]) for i in range(len(TOKEN[:-2]))]

In [297]:
TOKEN_2_GRAM[:10]

['吴京意淫', '意淫到', '到了', '了脑残', '脑残的', '的地步', '地步看', '看了', '了恶心', '恶心想']

In [298]:
words_count_2 = Counter(TOKEN_2_GRAM)

In [299]:
def prob_1(word):
    if words_count[word]==0:
        return 1 / len(TOKEN)/2 #出现次数为0的单词。设定概率为出现为1的一半
    return words_count[word] / len(TOKEN)

In [300]:
def prob_2(word1, word2):
    if word1 + word2 in words_count_2: return words_count_2[word1+word2] / len(TOKEN_2_GRAM)
    else:
        return 1 / len(TOKEN_2_GRAM)

In [301]:
prob_2('我们', '在')

2.0202836478241545e-05

In [302]:
prob_2('看', '电影')

0.00011048426199038344

In [303]:
#2.2 获得新的2-gram语言模型
def get_probability(sentence):
    words = cut(sentence)
    
    sentence_pro = 1
    
    for i, word in enumerate(words[:-1]):
        next_ = words[i+1]
        
        probability = prob_2(word, next_)/prob_1(word)
        #count(w1,w2)/count(w2)        
        sentence_pro *= probability
    
    return sentence_pro

In [305]:
get_probability('小明今天抽奖抽到一台飞机')

2.22108378102271e-08

In [306]:
comment_compared = [
    "今天晚上请你吃大餐，我们一起吃日料 明天晚上请你吃大餐，我们一起吃苹果",
    "真事一只好看的小猫 真是一只好看的小猫",
    "今晚我去吃火锅 今晚火锅去吃我",
    "洋葱奶昔来一杯 养乐多绿来一杯"
]

for s in comment_compared:
    s1, s2 = s.split()
    p1, p2 = get_probability(s1), get_probability(s2)
    
    better = s1 if p1 > p2 else s2
    
    print('{} is more possible'.format(better))
    print('-'*4 + ' {} with probility {}'.format(s1, p1))
    print('-'*4 + ' {} with probility {}'.format(s2, p2))

明天晚上请你吃大餐，我们一起吃苹果 is more possible
---- 今天晚上请你吃大餐，我们一起吃日料 with probility 5.589773310824115e-21
---- 明天晚上请你吃大餐，我们一起吃苹果 with probility 2.6830911891955763e-20
真是一只好看的小猫 is more possible
---- 真事一只好看的小猫 with probility 1.4840419116340077e-14
---- 真是一只好看的小猫 with probility 1.5056754580120136e-12
今晚我去吃火锅 is more possible
---- 今晚我去吃火锅 with probility 1.739333221620318e-08
---- 今晚火锅去吃我 with probility 3.784619239336749e-09
养乐多绿来一杯 is more possible
---- 洋葱奶昔来一杯 with probility 1.9778505986507345e-05
---- 养乐多绿来一杯 with probility 2.000001683570061


In [84]:
#3. 获得最优质的的语言
#生成n个句子，并能选择一个最合理的句子:

In [307]:
def generate_best(grammar,target,n): 
    sens=[]
    for i in range(n):
        sen=generate(create_grammar(grammar), target)
        pro=get_probability(sen)
        sens.append((sen,pro))
    sorted_sens=sorted(sens,key=lambda x :x[1],reverse=True)
    return sorted_sens

In [309]:
generate_best(host,'host',5)

[('您好我是53号客服,您需要订机票吗？', 1.5883836988955677e-20),
 ('您好我是8号客服,您需要人工服务吗？', 4.7989744460175026e-23),
 ('您好我是258号客服,您需要提建议吗？', 1.1177933620226865e-23),
 ('先生,您好我是18号客服,请问您要订机票吗？', 1.8977261203258143e-24),
 ('您好我是3号客服,请问您要提建议吗？', 1.6011221520948315e-25)]

这个模型有什么问题，你准备如何提升？
Ans:这个模型不能生成复杂的句子，用来计算概率的数据不够全面，只是针对电影的评论，对于生活化、工作化的语言不能很好的预测，如吃饭吃苹果的句子没有预测正确
提升：完善数据库

In [263]:
#基础理论部分

0. Can you come up out 3 sceneraies which use AI methods? <br>
ans: 自动驾驶，客服机器人，百度识图

1. How do we use Github; Why do we use Jupyter and Pycharm;<br>
ans:用Github来存储不同版本的文件，方便团队协作完成项目 <br>
用Jupyter比较方便记笔记，Pycharm可以写py文件，方便查看变量，适合于大型项目的开发

2. What's the Probability Model? <br>
ans:概率模型是用来描述不同随机变量之间关系的数学模型，<br>
通常情况下刻画了一个或多个随机变量之间的相互非确定性的概率关系.

3. Can you came up with some sceneraies at which we could use Probability Model? <br>
ans:自动答复，机器翻译，股票预测

4. Why do we use probability and what's the difficult points for programming based on parsing and pattern match? <br>
ans:1.因为可以基于大量的数据的统计特性提高智能识别的正确性
2.难点：如果要提升正确率，需要的数据量很大，运算慢
递归的运算方式很慢

5.What's the Language Model? <br>
ans: 计算句子出现概率的模型

6.Can you came up with some sceneraies at which we could use Language Model? <br>
Ans: 机器翻译、语音识别

7.What's the 1-gram language model? <br>
Ans: n-gram 语言模型的简化。一个句子(假设由四个词组成)出现的概率定义为$$ Pro(w_1 w_2 w_3 w_4) = Pr(w_1 | w_2 w_3 w_ 4) * P(w2 | w_3 w_4) * Pr(w_3 | w_4) * Pr(w_4)$$ <br>
就是每个词出现的概率与它前面n-1个词出现的先验概率的乘积，如果句子太长，这个模型的计算量非常大，而且离这个词越远的单词实际上对概率的影响越来越小，所以可以进行简化，最简单的模式就是1-gram，概率就被简化为$$ Pro(w_1 w_2 w_3 w_4) \sim Pr(w_1) * P(w_2) * Pr(w_3) * Pr(w_4)$$ 

8. What's the disadvantages and advantages of 1-gram language model; <br>
ans:优点是简单，缺点是准确度欠缺，损失了词语之间的关系信息

9. What't the 2-gram models;
ans:和1-gram一样，也是n-gram模型的简化，$$ Pro(w_1 w_2 w_3 w_4) \sim Pr(w_1 | w_2 ) * P(w2 | w_3 ) * Pr(w_3 | w_4) * Pr(w_4)$$ 